In [ ]:

# Here's a breakdown of the logic:

# [cite_start]The system will monitor Nifty 50 Spot and 6 Option Strikes (CE/PE – ATM, ITM, OTM) using 1-minute candles[cite: 3, 10]. [cite_start]It will compare the current 1-minute candle with the previous 1-minute candle for Nifty 50 Spot, CE (ATM, ITM, OTM), and PE (ATM, ITM, OTM)[cite: 3].

# [cite_start]There are two main parts, each with 4 sub-cases, and both parts must be checked for each strike (ATM, ITM, OTM) of both CE and PE[cite: 2].

# [cite_start]**Part 1 – Spot Breaks (Base Divergence Detection)** [cite: 4]
# [cite_start]This part detects when the Spot moves in one direction, but the option fails to follow[cite: 8].
# * [cite_start]**A1**: If Spot breaks HIGH, and CE (ATM/ITM/OTM) should break HIGH but *doesn’t* break HIGH, an alert is triggered[cite: 4].
# * [cite_start]**A2**: If Spot breaks HIGH, and PE (ATM/ITM/OTM) should break LOW but *doesn’t* break LOW, an alert is triggered[cite: 4].
# * [cite_start]**B1**: If Spot breaks LOW, and PE (ATM/ITM/OTM) should break HIGH but *doesn’t* break HIGH, an alert is triggered[cite: 4].
# * [cite_start]**B2**: If Spot breaks LOW, and CE (ATM/ITM/OTM) should break LOW but *doesn’t* break LOW, an alert is triggered[cite: 4].

# [cite_start]**Part 2 – Spot Fails (Reverse Divergence Detection)** [cite: 5, 6]
# [cite_start]This part detects when Nifty Spot fails to break either its previous HIGH or LOW, but options break their side, which is considered abnormal[cite: 5]. [cite_start]This means Spot stays still, and the option breaks the opposite side[cite: 8].
# * [cite_start]**C1**: If Spot fails to break HIGH, and CE (ATM/ITM/OTM) breaks HIGH, an alert is triggered for "CE is prematurely bullish"[cite: 6].
# * [cite_start]**C2**: If Spot fails to break LOW, and CE (ATM/ITM/OTM) breaks LOW, an alert is triggered for "CE is prematurely bearish"[cite: 6].
# * [cite_start]**C3**: If Spot fails to break LOW, and PE (ATM/ITM/OTM) breaks HIGH, an alert is triggered for "PE is prematurely bullish"[cite: 6].
# * [cite_start]**C4**: If Spot fails to break HIGH, and PE (ATM/ITM/OTM) breaks LOW, an alert is triggered for "PE is prematurely bearish"[cite: 6].

# [cite_start]In addition to the 8 divergence conditions, there are 2 system alerts (auto-reconnect + health monitoring alerts), making a total of 10 alert scenarios[cite: 7, 8].

# [cite_start]The system will include core features such as real-time tracking, 8 divergence logic conditions, desktop notifications with details (time, LTP, condition, strike), sound alerts, Excel logging (separate files for alerts, data\_log, errors, system\_log, false\_moves), auto-reconnect functionality, and a system monitor[cite: 9, 10]. [cite_start]An example of an alert format is also provided[cite: 11].

# ***

# **Confirmation to proceed:**

# Go Ahead

# ***

# **Python Code Structure (Illustrative):**

# This is a conceptual outline to demonstrate how you might structure the Python code based on the described logic. It does not include actual API calls or detailed error handling, which would be specific to the Upstox API and your desired implementation.

# ```python
import pandas as pd
import time
from datetime import datetime
import logging
import os

# --- Configuration ---
LOG_DIR = "logs"
ALERT_LOG_FILE = os.path.join(LOG_DIR, "alerts.xlsx")
DATA_LOG_FILE = os.path.join(LOG_DIR, "data_log.xlsx")
ERROR_LOG_FILE = os.path.join(LOG_DIR, "errors.xlsx")
SYSTEM_LOG_FILE = os.path.join(LOG_DIR, "system_log.xlsx")
FALSE_MOVES_LOG_FILE = os.path.join(LOG_DIR, "false_moves.xlsx")

# Ensure log directory exists
os.makedirs(LOG_DIR, exist_ok=True)

# Setup basic logging
logging.basicConfig(level=logging.INFO,
                    format='%(asctime)s - %(levelname)s - %(message)s',
                    handlers=[
                        logging.FileHandler(os.path.join(LOG_DIR, "app.log")),
                        logging.StreamHandler()
                    ])

# --- Data Structures (Example - In a real system, these would come from an API) ---
class CandleData:
    def __init__(self, high, low, close, ltp):
        self.high = high
        self.low = low
        self.close = close
        self.ltp = ltp

class MarketData:
    def __init__(self):
        self.nifty_spot_current_candle = None
        self.nifty_spot_previous_candle = None
        self.ce_atm_current_candle = None
        self.ce_atm_previous_candle = None
        self.ce_itm_current_candle = None
        self.ce_itm_previous_candle = None
        self.ce_otm_current_candle = None
        self.ce_otm_previous_candle = None
        self.pe_atm_current_candle = None
        self.pe_atm_previous_candle = None
        self.pe_itm_current_candle = None
        self.pe_itm_previous_candle = None
        self.pe_otm_current_candle = None
        self.pe_otm_previous_candle = None

    def update_data(self, new_spot_data, new_option_data):
        # This function would handle shifting current to previous and loading new data
        # For demonstration, we'll manually update
        pass

# --- Alerting and Logging Functions ---
def send_desktop_notification(alert_msg):
    # This would typically use a library like 'plyer' or 'win10toast'
    print(f"DESKTOP NOTIFICATION: {alert_msg}") # Placeholder
    logging.info(f"Desktop Notification Sent: {alert_msg}")

def play_sound_alert():
    # This would use a library like 'playsound' or 'pygame.mixer'
    print("SOUND ALERT!") # Placeholder
    logging.info("Sound Alert Triggered")

def log_alert_to_excel(alert_details):
    try:
        df = pd.DataFrame([alert_details])
        if not os.path.exists(ALERT_LOG_FILE):
            df.to_excel(ALERT_LOG_FILE, index=False)
        else:
            existing_df = pd.read_excel(ALERT_LOG_FILE)
            updated_df = pd.concat([existing_df, df], ignore_index=True)
            updated_df.to_excel(ALERT_LOG_FILE, index=False)
        logging.info(f"Alert logged to {ALERT_LOG_FILE}")
    except Exception as e:
        logging.error(f"Error logging alert to Excel: {e}")
        log_error(f"Error logging alert to Excel: {e}")

def log_data_to_excel(data_snapshot):
    try:
        df = pd.DataFrame([data_snapshot])
        if not os.path.exists(DATA_LOG_FILE):
            df.to_excel(DATA_LOG_FILE, index=False)
        else:
            existing_df = pd.read_excel(DATA_LOG_FILE)
            updated_df = pd.concat([existing_df, df], ignore_index=True)
            updated_df.to_excel(DATA_LOG_FILE, index=False)
        logging.debug(f"Data logged to {DATA_LOG_FILE}")
    except Exception as e:
        logging.error(f"Error logging data to Excel: {e}")
        log_error(f"Error logging data to Excel: {e}")

def log_error(error_msg):
    try:
        timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        df = pd.DataFrame([{"Timestamp": timestamp, "Error": error_msg}])
        if not os.path.exists(ERROR_LOG_FILE):
            df.to_excel(ERROR_LOG_FILE, index=False)
        else:
            existing_df = pd.read_excel(ERROR_LOG_FILE)
            updated_df = pd.concat([existing_df, df], ignore_index=True)
            updated_df.to_excel(ERROR_LOG_FILE, index=False)
        logging.error(f"Error logged: {error_msg}")
    except Exception as e:
        print(f"CRITICAL ERROR: Could not log error to file: {e}")

def log_system_status(status_msg):
    try:
        timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        df = pd.DataFrame([{"Timestamp": timestamp, "Status": status_msg}])
        if not os.path.exists(SYSTEM_LOG_FILE):
            df.to_excel(SYSTEM_LOG_FILE, index=False)
        else:
            existing_df = pd.read_excel(SYSTEM_LOG_FILE)
            updated_df = pd.concat([existing_df, df], ignore_index=True)
            updated_df.to_excel(SYSTEM_LOG_FILE, index=False)
        logging.info(f"System status logged: {status_msg}")
    except Exception as e:
        logging.error(f"Error logging system status: {e}")
        log_error(f"Error logging system status: {e}")

def log_false_move(details):
    try:
        df = pd.DataFrame([details])
        if not os.path.exists(FALSE_MOVES_LOG_FILE):
            df.to_excel(FALSE_MOVES_LOG_FILE, index=False)
        else:
            existing_df = pd.read_excel(FALSE_MOVES_LOG_FILE)
            updated_df = pd.concat([existing_df, df], ignore_index=True)
            updated_df.to_excel(FALSE_MOVES_LOG_FILE, index=False)
        logging.info(f"False move logged to {FALSE_MOVES_LOG_FILE}")
    except Exception as e:
        logging.error(f"Error logging false move to Excel: {e}")
        log_error(f"Error logging false move to Excel: {e}")

# --- Divergence Logic Functions ---

def check_part1_divergence(
    spot_current: CandleData, spot_previous: CandleData,
    option_current: CandleData, option_previous: CandleData,
    strike_type: str, option_type: str # e.g., "ATM", "CE"
):
    alerts = []
    current_time = datetime.now().strftime("%H:%M:%S")

    # Spot breaks HIGH
    if spot_current.high > spot_previous.high:
        # A1: Spot breaks HIGH, CE should break HIGH, Doesn't break HIGH
        if option_type == "CE" and option_current.high <= option_previous.high:
            alert_msg = f"[ALERT] A1 – Spot High Break, CE Fails to Follow"
            alert_details = {
                "Time": current_time,
                "LTP": option_current.ltp,
                "Condition": alert_msg,
                "Strike": f"{strike_type} {option_type}",
                "Spot": f"{spot_current.ltp} (Broke HIGH)",
                "Option Prev High": option_previous.high,
                "Option Current High": option_current.high
            }
            alerts.append(alert_details)
        # A2: Spot breaks HIGH, PE should break LOW, Doesn't break LOW
        elif option_type == "PE" and option_current.low >= option_previous.low:
            alert_msg = f"[ALERT] A2 – Spot High Break, PE Fails to Follow"
            alert_details = {
                "Time": current_time,
                "LTP": option_current.ltp,
                "Condition": alert_msg,
                "Strike": f"{strike_type} {option_type}",
                "Spot": f"{spot_current.ltp} (Broke HIGH)",
                "Option Prev Low": option_previous.low,
                "Option Current Low": option_current.low
            }
            alerts.append(alert_details)

    # Spot breaks LOW
    if spot_current.low < spot_previous.low:
        # B1: Spot breaks LOW, PE should break HIGH, Doesn't break HIGH
        if option_type == "PE" and option_current.high <= option_previous.high:
            alert_msg = f"[ALERT] B1 – Spot Low Break, PE Fails to Follow"
            alert_details = {
                "Time": current_time,
                "LTP": option_current.ltp,
                "Condition": alert_msg,
                "Strike": f"{strike_type} {option_type}",
                "Spot": f"{spot_current.ltp} (Broke LOW)",
                "Option Prev High": option_previous.high,
                "Option Current High": option_current.high
            }
            alerts.append(alert_details)
        # B2: Spot breaks LOW, CE should break LOW, Doesn't break LOW
        elif option_type == "CE" and option_current.low >= option_previous.low:
            alert_msg = f"[ALERT] B2 – Spot Low Break, CE Fails to Follow"
            alert_details = {
                "Time": current_time,
                "LTP": option_current.ltp,
                "Condition": alert_msg,
                "Strike": f"{strike_type} {option_type}",
                "Spot": f"{spot_current.ltp} (Broke LOW)",
                "Option Prev Low": option_previous.low,
                "Option Current Low": option_current.low
            }
            alerts.append(alert_details)
    return alerts

def check_part2_divergence(
    spot_current: CandleData, spot_previous: CandleData,
    option_current: CandleData, option_previous: CandleData,
    strike_type: str, option_type: str # e.g., "ATM", "CE"
):
    alerts = []
    current_time = datetime.now().strftime("%H:%M:%S")

    # Spot fails to break HIGH and Spot fails to break LOW (Spot stays still)
    spot_fails_high = spot_current.high <= spot_previous.high
    spot_fails_low = spot_current.low >= spot_previous.low

    # C1: Spot fails to break HIGH, CE Breaks HIGH
    if option_type == "CE" and spot_fails_high and option_current.high > option_previous.high:
        alert_msg = f"[ALERT] C1 – Abnormal CE Bullish Move"
        alert_details = {
            "Time": current_time,
            "LTP": option_current.ltp,
            "Condition": alert_msg,
            "Strike": f"{strike_type} {option_type}",
            "Spot": f"{spot_current.ltp} (Did NOT break HIGH)",
            "Prev High": option_previous.high,
            "Current High": option_current.high
        }
        alerts.append(alert_details)
    # C2: Spot fails to break LOW, CE Breaks LOW
    elif option_type == "CE" and spot_fails_low and option_current.low < option_previous.low:
        alert_msg = f"[ALERT] C2 – Abnormal CE Bearish Move"
        alert_details = {
            "Time": current_time,
            "LTP": option_current.ltp,
            "Condition": alert_msg,
            "Strike": f"{strike_type} {option_type}",
            "Spot": f"{spot_current.ltp} (Did NOT break LOW)",
            "Prev Low": option_previous.low,
            "Current Low": option_current.low
        }
        alerts.append(alert_details)
    # C3: Spot fails to break LOW, PE Breaks HIGH
    elif option_type == "PE" and spot_fails_low and option_current.high > option_previous.high:
        alert_msg = f"[ALERT] C3 – Abnormal PE Bullish Move"
        alert_details = {
            "Time": current_time,
            "LTP": option_current.ltp,
            "Condition": alert_msg,
            "Strike": f"{strike_type} {option_type}",
            "Spot": f"{spot_current.ltp} (Did NOT break LOW)",
            "Prev High": option_previous.high,
            "Current High": option_current.high
        }
        alerts.append(alert_details)
    # C4: Spot fails to break HIGH, PE Breaks LOW
    elif option_type == "PE" and spot_fails_high and option_current.low < option_previous.low:
        alert_msg = f"[ALERT] C4 – Abnormal PE Bearish Move"
        alert_details = {
            "Time": current_time,
            "LTP": option_current.ltp,
            "Condition": alert_msg,
            "Strike": f"{strike_type} {option_type}",
            "Spot": f"{spot_current.ltp} (Did NOT break HIGH)",
            "Prev Low": option_previous.low,
            "Current Low": option_current.low
        }
        alerts.append(alert_details)
    return alerts

# --- Main System Loop ---
def run_divergence_system():
    market_data = MarketData()
    # In a real scenario, you'd initialize market_data with current and previous candles
    # by fetching historical 1-min data from Upstox API or similar.

    # Dummy data for demonstration:
    # Spot moves up, CE fails to follow (A1 type scenario)
    market_data.nifty_spot_previous_candle = CandleData(high=24500, low=24400, close=24480, ltp=24480)
    market_data.nifty_spot_current_candle = CandleData(high=24550, low=24490, close=24540, ltp=24540)
    market_data.ce_atm_previous_candle = CandleData(high=100, low=90, close=95, ltp=95)
    market_data.ce_atm_current_candle = CandleData(high=98, low=92, close=96, ltp=96) # Should have broken HIGH (>100) but didn't

    # Spot fails to break LOW, PE breaks HIGH (C3 type scenario)
    market_data.nifty_spot_previous_candle_c3 = CandleData(high=24550, low=24500, close=24520, ltp=24520)
    market_data.nifty_spot_current_candle_c3 = CandleData(high=24540, low=24510, close=24530, ltp=24530) # Fails to break LOW (24500)
    market_data.pe_atm_previous_candle_c3 = CandleData(high=60, low=50, close=55, ltp=55)
    market_data.pe_atm_current_candle_c3 = CandleData(high=62.40, low=52, close=61.80, ltp=62.10) # Breaks HIGH (>60)

    log_system_status("ONLINE")
    logging.info("Divergence system started.")

    while True:
        try:
            # 1. Fetch real-time 1-minute candle data for Nifty Spot and all 6 options
            # This would involve API calls to Upstox.
            # For this example, we'll use our dummy data to trigger alerts.
            current_nifty_spot = market_data.nifty_spot_current_candle
            previous_nifty_spot = market_data.nifty_spot_previous_candle

            current_ce_atm = market_data.ce_atm_current_candle
            previous_ce_atm = market_data.ce_atm_previous_candle

            # Simulate for C3 scenario
            current_nifty_spot_c3 = market_data.nifty_spot_current_candle_c3
            previous_nifty_spot_c3 = market_data.nifty_spot_previous_candle_c3
            current_pe_atm_c3 = market_data.pe_atm_current_candle_c3
            previous_pe_atm_c3 = market_data.pe_atm_previous_candle_c3


            # Log all current data (optional, but good for debugging and history)
            # log_data_to_excel({
            #     "Timestamp": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
            #     "NiftySpotLTP": current_nifty_spot.ltp,
            #     "CE_ATM_LTP": current_ce_atm.ltp,
            #     # ... include all other LTPs and candle data
            # })

            # 2. Apply Part 1 Divergence Logic
            part1_alerts = []
            part1_alerts.extend(check_part1_divergence(current_nifty_spot, previous_nifty_spot, current_ce_atm, previous_ce_atm, "ATM", "CE"))
            # You would repeat for ITM, OTM for CE and PE

            for alert in part1_alerts:
                print(f"Triggered Alert: {alert['Condition']}")
                send_desktop_notification(f"{alert['Condition']} - {alert['Strike']} @ {alert['LTP']}")
                play_sound_alert()
                log_alert_to_excel(alert)

            # 3. Apply Part 2 Divergence Logic
            part2_alerts = []
            # Example for C3 from the document:
            part2_alerts.extend(check_part2_divergence(current_nifty_spot_c3, previous_nifty_spot_c3, current_pe_atm_c3, previous_pe_atm_c3, "ATM", "PE"))
            # You would repeat for all 8 cases of Part 2 across all strike types

            for alert in part2_alerts:
                print(f"Triggered Alert: {alert['Condition']}")
                send_desktop_notification(f"{alert['Condition']} - {alert['Strike']} @ {alert['LTP']}")
                play_sound_alert()
                log_alert_to_excel(alert)

            # 4. Implement Auto-Reconnect & System Monitor (Placeholder)
            # This would involve checking API connection status.
            # If disconnected, attempt reconnection and log to system_log and error_log.
            # system_monitor_status = "ONLINE" if is_connected_to_upstox() else "OFFLINE"
            # log_system_status(f"Status: {system_monitor_status}, Current Time: {datetime.now().strftime('%H:%M:%S')}, NiftyLTP: {current_nifty_spot.ltp}, CE_ATMLTP: {current_ce_atm.ltp}")


            time.sleep(60) # Check every 60 seconds for 1-minute candle completion
            # In a real system, you'd likely use a websocket for real-time tick data
            # and aggregate it into 1-minute candles.

            # To demonstrate a single run and exit for this example
            break

        except Exception as e:
            logging.error(f"An unhandled error occurred: {e}")
            log_error(f"Main loop error: {e}")
            # Potentially attempt to re-establish connection or pause and retry
            time.sleep(30) # Wait before retrying after an error

# To run the system (uncomment to test):
# if __name__ == "__main__":
#     run_divergence_system()

```





import pandas as pd
import math
from datetime import datetime, timedelta

# --- Configuration (Placeholders) ---
# Replace with your actual API client import and authentication details
# from upstox_api import UpstoxClient # Example for Upstox
# from zerodha_api import KiteConnect # Example for Zerodha

# API related placeholders
API_KEY = "YOUR_API_KEY"
API_SECRET = "YOUR_API_SECRET"
ACCESS_TOKEN = "YOUR_ACCESS_TOKEN" # This typically needs to be generated via an OAuth flow

# --- Helper Functions (Mock for demonstration) ---

def get_current_nifty_spot_price():
    """
    Mocks fetching the current Nifty 50 spot price.
    In a real scenario, this would be an API call to get real-time LTP.
    """
    print("Fetching current Nifty 50 spot price...")
    # Replace with actual API call, e.g., upstox_client.get_ltp('NSE_INDEX|Nifty 50')
    return 24550.0 # Example Nifty Spot Price 

def get_instrument_master(exchange="NSE_FO"):
    """
    Mocks fetching the instrument master for Nifty options.
    In a real scenario, this would involve downloading a file or calling an API endpoint.
    This example generates a synthetic instrument master for Nifty 50 options.
    """
    print(f"Fetching instrument master for {exchange}...")
    instruments = []
    base_symbol = "NIFTY"
    current_year = datetime.now().year
    # For demonstration, let's assume monthly expiries for next few months
    expiries = []
    # Get nearest weekly expiry (approximate, real API would give exact date)
    today = datetime.now()
    # Find next Thursday (weekly expiry for Nifty)
    days_until_thursday = (3 - today.weekday() + 7) % 7 # Thursday is weekday 3
    next_thursday = today + timedelta(days=days_until_thursday)
    expiries.append(next_thursday.strftime("%y%b").upper() if next_thursday.month != today.month else next_thursday.strftime("%y%b%d").upper()) # YYMON (e.g., 25JUN) or YYMON_DD (e.g., 25JUN06)

    # Add a few monthly expiries (last Thursday of the month)
    for i in range(3): # Next 3 months
        # This is a simplified way to get month end, actual expiry date logic is complex
        month_end = (today.replace(day=1) + timedelta(days=32)).replace(day=1) - timedelta(days=1)
        expiries.append(month_end.strftime("%y%b").upper()) # e.g., 25JUL, 25AUG

    strike_increment = 50 # Nifty options typically have 50 point increments

    # Generate a range of strikes around a hypothetical spot price for testing
    # In a real scenario, you'd fetch the actual available strikes from your API's instrument master
    hypothetical_spot_center = 24500
    min_strike = hypothetical_spot_center - 10 * strike_increment
    max_strike = hypothetical_spot_center + 10 * strike_increment

    for expiry in expiries:
        for strike in range(min_strike, max_strike + strike_increment, strike_increment):
            # CE
            instruments.append({
                "instrument_token": f"TOKEN_CE_{strike}_{expiry}",
                "tradingsymbol": f"{base_symbol}{expiry}{strike}CE",
                "instrument_type": "CE",
                "strike_price": float(strike),
                "expiry_date": expiry,
                "lot_size": 50,
                "exchange": exchange
            })
            # PE
            instruments.append({
                "instrument_token": f"TOKEN_PE_{strike}_{expiry}",
                "tradingsymbol": f"{base_symbol}{expiry}{strike}PE",
                "instrument_type": "PE",
                "strike_price": float(strike),
                "expiry_date": expiry,
                "lot_size": 50,
                "exchange": exchange
            })
    return pd.DataFrame(instruments)

# --- Core Logic for Strike Calculation ---

def get_nearest_expiry(instrument_master_df):
    """
    Determines the nearest expiry date from the instrument master.
    Assumes expiry_date is in YYMON or YYMONDD format (e.g., 25JUN, 25JUN06).
    """
    today = datetime.now()
    parsed_expiries = []

    for idx, row in instrument_master_df.iterrows():
        expiry_str = str(row['expiry_date'])
        try:
            # Try to parse as YYMONDD (e.g., 25JUN06)
            expiry_date = datetime.strptime(expiry_str, "%y%b%d").replace(year=2000 + int(expiry_str[:2]))
        except ValueError:
            try:
                # Try to parse as YYMON (e.g., 25JUN) - assuming last day of month if no day specified
                # This is a simplification; real options expire on specific Thursdays
                expiry_date = datetime.strptime(expiry_str, "%y%b").replace(year=2000 + int(expiry_str[:2]))
                # For monthly options, assume last Thursday, but here just setting to month end for simplicity
                # This needs refinement for actual expiry logic.
                month_end = (expiry_date.replace(day=1) + timedelta(days=32)).replace(day=1) - timedelta(days=1)
                expiry_date = month_end
            except ValueError:
                continue # Skip unparseable dates

        if expiry_date >= today:
            parsed_expiries.append((expiry_date, expiry_str))

    if not parsed_expiries:
        return None, None # No future expiries found

    parsed_expiries.sort(key=lambda x: x[0])
    nearest_date_obj, nearest_date_str = parsed_expiries[0]
    return nearest_date_str, nearest_date_obj # Return the string as it appears in instrument_master, and the datetime object

def get_option_strikes(nifty_spot_ltp: float, instrument_master_df: pd.DataFrame, expiry_str: str, num_strikes_each_side: int = 2):
    """
    Calculates ATM, ITM, OTM strikes for CE and PE for a given expiry.
    num_strikes_each_side determines how many ITM/OTM strikes to return beyond ATM.
    """
    if expiry_str is None:
        print("No valid nearest expiry found.")
        return {}

    # Filter instruments for the nearest expiry
    filtered_instruments = instrument_master_df[instrument_master_df['expiry_date'] == expiry_str]

    if filtered_instruments.empty:
        print(f"No instruments found for expiry: {expiry_str}")
        return {}

    # Get unique sorted strike prices for the given expiry
    available_strikes = sorted(filtered_instruments['strike_price'].unique())

    if not available_strikes:
        print(f"No strike prices available for expiry: {expiry_str}")
        return {}

    # Find ATM strike
    atm_strike = min(available_strikes, key=lambda x: abs(x - nifty_spot_ltp))

    # Determine ATM for CE and PE
    atm_ce_strike = atm_strike
    atm_pe_strike = atm_strike

    # Get ITM/OTM strikes
    strikes = {
        "CE": {"ATM": atm_ce_strike, "ITM": [], "OTM": []},
        "PE": {"ATM": atm_pe_strike, "ITM": [], "OTM": []}
    }

    atm_index = available_strikes.index(atm_strike)

    # For CE: ITM are lower strikes, OTM are higher strikes
    # For PE: ITM are higher strikes, OTM are lower strikes

    # Add ITM and OTM strikes based on num_strikes_each_side
    for i in range(1, num_strikes_each_side + 1):
        # CE ITM (lower strikes)
        if atm_index - i >= 0:
            strikes["CE"]["ITM"].append(available_strikes[atm_index - i])
        # PE ITM (higher strikes)
        if atm_index + i < len(available_strikes):
            strikes["PE"]["ITM"].append(available_strikes[atm_index + i])

        # CE OTM (higher strikes)
        if atm_index + i < len(available_strikes):
            strikes["CE"]["OTM"].append(available_strikes[atm_index + i])
        # PE OTM (lower strikes)
        if atm_index - i >= 0:
            strikes["PE"]["OTM"].append(available_strikes[atm_index - i])

    # Sort ITM/OTM strikes to ensure consistent order
    strikes["CE"]["ITM"].sort(reverse=True) # Highest ITM first
    strikes["PE"]["ITM"].sort() # Lowest ITM first
    strikes["CE"]["OTM"].sort() # Lowest OTM first
    strikes["PE"]["OTM"].sort(reverse=True) # Highest OTM first

    return strikes

# --- Main Execution ---

def fetch_and_display_strikes():
    print("Starting strike fetching process...")

    # 1. Get current Nifty Spot Price
    nifty_spot_ltp = get_current_nifty_spot_price()
    print(f"Current Nifty 50 Spot LTP: {nifty_spot_ltp}")

    # 2. Get Instrument Master (should be done once or infrequently, then cached)
    instrument_master_df = get_instrument_master()
    if instrument_master_df.empty:
        print("Could not retrieve instrument master. Exiting.")
        return

    # 3. Determine Nearest Expiry
    nearest_expiry_str, nearest_expiry_date_obj = get_nearest_expiry(instrument_master_df)
    if not nearest_expiry_str:
        print("No suitable expiry found. Exiting.")
        return
    print(f"Nearest Expiry: {nearest_expiry_str} ({nearest_expiry_date_obj.strftime('%Y-%m-%d')})")

    # 4. Calculate ATM, ITM, OTM strikes for both CE and PE for the nearest expiry
    # We want ATM, ITM, OTM, so let's get 1 ITM and 1 OTM for each side
    num_strikes_to_fetch = 1 # This will give us ATM, 1 ITM, 1 OTM for each type

    option_strikes = get_option_strikes(
        nifty_spot_ltp,
        instrument_master_df,
        nearest_expiry_str,
        num_strikes_each_side=num_strikes_to_fetch
    )

    if not option_strikes:
        print("Failed to calculate option strikes.")
        return

    print("\n--- Current Option Strikes ---")
    for option_type, categories in option_strikes.items():
        print(f"  {option_type} (Expiry: {nearest_expiry_str}):")
        print(f"    ATM: {categories['ATM']}")
        print(f"    ITM: {categories['ITM']}")
        print(f"    OTM: {categories['OTM']}")

    print("\nProcess completed.")

# To run the strike fetching system:
if __name__ == "__main__":
    fetch_and_display_strikes()

In [2]:
data = "https://assets.upstox.com/market-quote/instruments/exchange/complete.csv.gz"
import pandas as pd

In [ ]:
def new (strike,expiry):
    return pd.read_csv(data, compression='gzip').query(f"name == 'NIFTY' and strike=={strike} and expiry == '{expiry}'")['instrument_key']


In [14]:
new(25000, '2025-06-26')  # Example usage to get the instrument key for NIFTY 24500 CE for expiry 25JUN

'NSE_FO|71381'